In [63]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from metsim import MetSim
import xarray as xr

from synoptic.services import stations_timeseries
from synoptic.services import stations_nearesttime, stations_latest

In [50]:
# To keep things tidy, put the function params in the params variable
params = dict(
    stid="samw1",
    vars=["air_temp", "precip_accum", 'snow_water_equiv'],
    start=datetime(2023, 7, 1),
    end=datetime(2024, 4, 1),
)

a = stations_timeseries(verbose="HIDE", **params)


 🚚💨 Speedy Delivery from Synoptic API [timeseries]: https://api.synopticdata.com/v2/stations/timeseries?stid=samw1&vars=air_temp,precip_accum,snow_water_equiv&start=202307010000&end=202404010000&token=🙈HIDDEN



In [51]:
from pytz import UTC
# Interpolate the missing values
a.interpolate(inplace=True)

# Seperate the data into two dataframes, before and after October 1
a1 = a.loc[datetime(2023, 7, 3).replace(tzinfo=UTC):datetime(2023, 10, 1).replace(tzinfo=UTC)]
a2 = a.loc[datetime(2023, 10, 1).replace(tzinfo=UTC):]






In [55]:
# Copy the dataframe a2 to a2_copy
a2_copy = a2.copy()

# Create a mask to identify rows where the index is less than or equal to October 2, 2023
mask = a2_copy.index <= datetime(2023, 10, 2).replace(tzinfo=UTC)

# Set the 'precip_accum' column to 0 for rows that satisfy the mask condition
a2_copy.loc[mask, 'precip_accum'] = 0

# Update the value of a2 to the modified copy
a2 = a2_copy

# Calculate the difference between the maximum value of 'precip_accum' and the previous value
a2['precip_accum'] = a2['precip_accum'].cummax().diff()


## Generate SW from MetSim

In [66]:
dates = pd.date_range('10/1/2023', '4/1/2024')
shape = (len(dates), 1, 1, )
dims = ('time', 'lat', 'lon', )

# We are running only one site, at these coordinates
lats = [a1.attrs['latitude']]
lons = [a1.attrs['longitude']]
elev = [a1.attrs['ELEVATION']/3.281] # meters
coords = {'time': dates, 'lat': lats, 'lon': lons}

# Create the initial met data input data structure
met_data = xr.Dataset(coords=coords)

# Initialize data array with needed variables
for varname in ['prec', 't_min', 't_max']:
    met_data[varname] = xr.DataArray(data=np.full(shape, np.nan),
                                     coords=coords, dims=dims,
                                     name=varname)

In [61]:
# data = pd.read_csv('./salmon_cleaned_WY23.csv')
# data = data[data['Date_Time'] < '2023-09-01'] 
# data['air_temp_set_1'] = data['air_temp_set_1'].interpolate(method='linear')
# data['precip_accum_set_1'] = data['precip_accum_set_1'].interpolate(method='linear')
# data.loc[data['precip_accum_set_1'] > 1100, 'precip_accum_set_1'] = 0

# data['time'] = pd.to_datetime(data.time)

# data['precip_accum_set_1'] = data['precip_accum_set_1'].cummax()

# Set the 'datetime' column as the index of the dataframe
# data.set_index('time', inplace=True)

prec_vals = a2['accppt'].resample('D').sum()
print(prec_vals.max())

# Resample the data to daily frequency and calculate the maximum and minimum temperatures
tmax_vals = data['airtemp'].resample('D').max()
tmin_vals = data['airtemp'].resample('D').min()

# Interpolate the temperature values to fill in any missing days
tmax_vals = tmax_vals.interpolate(method='linear')
tmin_vals = tmin_vals.interpolate(method='linear')

met_data['prec'].values[:, 0, 0] = prec_vals

# Assign the daily maximum and minimum temperatures to the met_data xarray, converting to Celsius
met_data['t_min'].values[:, 0, 0] = tmin_vals - 273.15
met_data['t_max'].values[:, 0, 0] = tmax_vals - 273.15

# met_data.to_netcdf('./input/rc_forcing.nc')
met_data

-119.8383


In [67]:
a2

,air_temp,precip_accum,snow_water_equiv
date_time,,,
2023-10-01 00:00:00+00:00,9.4,NaN,5.08
2023-10-01 01:00:00+00:00,7.8,0.0,5.08
2023-10-01 02:00:00+00:00,4.9,0.0,5.08
2023-10-01 03:00:00+00:00,6.1,0.0,5.08
2023-10-01 04:00:00+00:00,5.8,0.0,5.08
...,...,...,...
2024-03-31 20:00:00+00:00,6.4,0.0,170.18
2024-03-31 21:00:00+00:00,7.6,0.0,167.64
2024-03-31 22:00:00+00:00,9.1,0.0,167.64
